In [1]:
from collections import defaultdict
import Geometry as gm

SCALE = 25
DEVIATION = 10

RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
PURPLE = (255, 0, 255)
CYAN = (0, 255, 255)
YELLOW = (255, 255, 0)

def def_value():
    return None

layers = []
n = 5
for i in range(n):
    layers.append(gm.DCEL())
    #Temporal dictionaries to store the half-edges while building the layer
    edges = dict()
    vertices = dict()
    faces = dict()
    #Reads input file
    file_name = f"inputs/layer0{i + 1}"
    
    #Ignores overhead of the files
    ari = open(file_name + ".aristas").readlines()[4:]
    vert = open(file_name + ".vertices").readlines()[4:]
    car = open(file_name + ".caras").readlines()[4:]
    
    #Start building the layer from the vertices
    for line in vert:
        name, x, y, incident = line.split()
        vertices[name] = gm.Vertex(float(x) * SCALE + DEVIATION, float(y)*SCALE + DEVIATION)
        #We don't have half-edges so we don't add incident edge yet
    #Start making the half-edges
    for line in ari:
        name, ori, twin, face, next, prev = line.split()
        #Create based on origin point
        edges[name] = gm.Half_Edge(vertices[ori])
    #Add incident edges
    for line in vert:
        name, x, y, incident = line.split()
        vertices[name].incident = incident
        layers[i].add_vertex(vertices[name])
    #Create the faces
    for line in car:
        name, internal, external = line.split()
        internal = internal.replace("[", "").replace("]", "")
        external = external.replace("[", "").replace("]", "")
        icomp = []
        if len(internal.split(",")) >= 2:
            internal = internal.split(",")
            for comp in internal:
                icomp.append(edges[comp])
        if internal == "None": internal = None
        if external == "None": external = None
        # Adds pointers to different cycles
        if icomp:
            faces[name] = gm.Face(external, icomp)
        else:
            faces[name] = gm.Face(external, internal)
        layers[i].add_face(faces[name])
    #Complete information about the half-edges
    for line in ari:
        name, ori, twin, face, next, prev = line.split()
        #Create based on origin point
        edges[name].twin = edges[twin]
        edges[name].next = edges[next]
        edges[name].prev = edges[prev]
        edges[name].face = faces[face]
        layers[i].add_half_edge(edges[name])
    
intersections = gm.calcIntersections(layers)

[(222.8048780487805, 222.8048780487805), (260.0000000000073, 187.3809523809455), (213.9622641509434, 231.22641509433961), (210.0, 235.0), (262.5, 185.0), (222.8048780487805, 222.8048780487805), (260.0000000000073, 187.3809523809455), (213.9622641509434, 231.22641509433961), (210.0, 235.0), (262.5, 185.0), (335.0000000000062, 210.00000000000213), (335.0000000000062, 210.00000000000213), (335.0, 272.5), (335.0, 272.5), (235.0, 285.0), (235.0, 285.0), (210.000000000005, 59.999999999995), (210.0, 110.0), (210.000000000005, 60.0000000000025), (210.000000000005, 59.999999999995), (210.0, 110.0), (210.000000000005, 60.0000000000025), (210.0, 60.0), (260.0000000000175, 85.00000000000875), (260.0, 85.0), (210.0, 60.0), (260.0000000000175, 85.00000000000875), (260.0, 85.0), (260.0000000000175, 84.99999999999125), (260.0000000000175, 84.99999999999125), (135.0, 135.0), (85.00000000001, 135.0), (135.0000000000075, 135.0), (135.0, 135.0), (85.00000000001, 135.0), (135.0000000000075, 135.0), (135.00

In [2]:
import pygame

# Helper function to draw in pygame screen
def draw_point(screen, vertex: gm.Vertex, color):
    pygame.draw.circle(screen, color, vertex.coord, 2)

def draw_edge(screen, edge: gm.Half_Edge, color):
    pygame.draw.line(screen, color, edge.origin.coord, edge.twin.origin.coord)
    
def draw_layer(screen, layer: gm.DCEL, color):
    edges = layer.half_edges
    points = layer.vertices
    #for point in points:
        #draw_point(screen, point, (0, 0, 0))
    for edge in edges:
        draw_edge(screen, edge, (0, 0, 255))
    for intersection in intersections:
        pygame.draw.circle(screen, (0, 0, 0), (intersection.x,intersection.y), 2)

pygame 2.4.0 (SDL 2.26.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pygame.init()
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
screen.fill((255, 255, 255))

for layer in layers:
    draw_layer(screen, layer, (255, 255, 255))

pygame.display.update()

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            quit()


error: video system not initialized

: 